# [PREDICTING MOVIE REVENUE (IMDB)](https://www.kaggle.com/PromptCloudHQ/imdb-data)

### Given a data set of the 1,000 most popular movies on IMDB in the last 10 years, can you predict the revenue of a given movie?

Create a user for kaggle if not already done.
<br><br>



### Questions/Exercises:

**1.** Download the data programatically with python. 

(**Hint:** To generate your own kaggle.json file go to *your profile* -> *account* -> *Create new API Token*, and then place the file it in ~/.kaggle/)


In [1]:
#Using kaggle to downloading and unzipping the data
import kaggle

kaggle.api.authenticate()
kaggle.api.dataset_download_files('PromptCloudHQ/imdb-data', path='./', unzip=True)

You will now have to make a train-test-split of the data so that you have a way of verifying the model later. (We suggest 2/3 for training and 1/3 for testing)
	
Here is a link to the [train-test-split documentation for sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

**2.** Fit the training-data into a model, present it visually and reflect on wether it is a good fit or not.


In [2]:
#Loading the csv data into pandas dataframe
import pandas as pd

df = pd.read_csv("IMDB-Movie-Data.csv")

In [3]:
#Removing the revenue and other non-wanted features from the dataframe
%matplotlib notebook
import sklearn

df = df.drop(["Rank", "Title", "Description", "Director","Actors", "Rating", "Votes"], 1)

In [4]:
#Since genre is a string of genres seperated by a comma, we have to convert it to pd.Series
serie = pd.Series([genre_string.split(",") for genre_string in df["Genre"]])

#Now when the genres are converted to Series, we can make the dummies
genre_dummies = pd.get_dummies(serie.apply(pd.Series).stack()).sum(level=0)

In [5]:
#Getting the top genres
genre_counts = {}

for genre in genre_dummies.columns:
    count = 0
    for line in genre_dummies[genre]:
        if line != 0:
            count += 1 
    genre_counts[genre] = count

sorted_genres = sorted(genre_counts.items(), key=lambda kv: -kv[1])
genre_keys = list(dict(sorted_genres[:9]).keys())
genre_keys

['Drama',
 'Action',
 'Comedy',
 'Adventure',
 'Thriller',
 'Crime',
 'Romance',
 'Sci-Fi',
 'Horror']

In [6]:
#Setting the non-top genres to other
genre_dummies["Other genres"] = 0
genre_keys.append("Other genres")

for genre in genre_dummies.columns:
    if(genre not in genre_keys):
        genre_dummies["Other genres"] += genre_dummies[genre]
        genre_dummies = genre_dummies.drop(genre, 1)

#Setting Other genre values to binary value
def set_value(key):
    genre_dummies["Other genres"][key] = 1   
z = [set_value(key) for key, value in enumerate(genre_dummies["Other genres"]) if value > 1]

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [7]:
#Now we have vectors that can hold multiple dimentions of genres.
genre_dummies.head()

,Action,Adventure,Comedy,Crime,Drama,Horror,Romance,Sci-Fi,Thriller,Other genres
0,1,1,0,0,0,0,0,1,0,0
1,0,1,0,0,0,0,0,1,0,1
2,0,0,0,0,0,1,0,0,1,0
3,0,0,1,0,0,0,0,0,0,1
4,1,1,0,0,0,0,0,0,0,1


In [8]:
#Creating a copy of df without the genre column
df_no_genre = df.drop("Genre", 1)

In [9]:
df_no_genre.head()

,Year,Runtime (Minutes),Revenue (Millions),Metascore
0,2014,121,333.13,76.0
1,2012,124,126.46,65.0
2,2016,117,138.12,62.0
3,2016,108,270.32,59.0
4,2016,123,325.02,40.0


In [10]:
#Merging the two df of vectors into one
vector_df = pd.merge(df_no_genre, genre_dummies, on=df.index).drop("key_0",1)

In [11]:
#Using the Imputer to replace NaN's by setting them to the median value
from sklearn.impute import SimpleImputer

NaN_imp = SimpleImputer(strategy="median")
NaN_imp.fit(vector_df)

vector_df = pd.DataFrame(NaN_imp.transform(vector_df), columns=vector_df.columns)

In [13]:
#making sure that all data is ints and not floats
from sklearn import preprocessing

lab_enc = preprocessing.LabelEncoder()

for column in vector_df.columns:
    vector_df[column] = lab_enc.fit_transform(vector_df[column])

In [14]:
vector_df.head()

,Year,Runtime (Minutes),Revenue (Millions),Metascore,Action,Adventure,Comedy,Crime,Drama,Horror,Romance,Sci-Fi,Thriller,Other genres
0,8,43,783,61,1,1,0,0,0,0,0,1,0,0
1,6,46,623,50,0,1,0,0,0,0,0,1,0,1
2,10,39,646,47,0,0,0,0,0,1,0,0,1,0
3,10,30,761,43,0,0,1,0,0,0,0,0,0,1
4,10,45,781,24,1,1,0,0,0,0,0,0,0,1


In [15]:
import numpy as np

X = vector_df.drop(["Revenue (Millions)"], 1)
y = np.array(vector_df["Revenue (Millions)"])

In [16]:
X.head()

,Year,Runtime (Minutes),Metascore,Action,Adventure,Comedy,Crime,Drama,Horror,Romance,Sci-Fi,Thriller,Other genres
0,8,43,61,1,1,0,0,0,0,0,1,0,0
1,6,46,50,0,1,0,0,0,0,0,1,0,1
2,10,39,47,0,0,0,0,0,1,0,0,1,0
3,10,30,43,0,0,1,0,0,0,0,0,0,1
4,10,45,24,1,1,0,0,0,0,0,0,0,1


In [17]:
#Train-Test-Splitting the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=4)

In [18]:
#Now we have two datasets, 3/4 of the data is for training, and 1/4 is for testing the model (selected randomly)
print("training-data: ", X_train.shape, " y:", y_train.shape)
print("test-data:     ", X_test.shape, " y:", y_test.shape)

training-data:  (750, 13)  y: (750,)
test-data:      (250, 13)  y: (250,)


In [19]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [20]:
from sklearn.metrics import accuracy_score
y_pred = logreg.predict(X_test)

print("Model Accuracy:",accuracy_score(y_test, y_pred) * 100, "%")

Model Accuracy: 10.0 %


**3.** Now take the test-data, test the model and evaluate how accurate the model is.

### Review Questions
1. Was the data downloaded programatically with python?
2. Was the data fitted into the model, presented visually and reflected on?
3. Was the revenue predicted using the model for the given metascores?